In [2]:
#include "stdio.h"
#include "stdlib.h"
#include <iostream>
#include <vector>

/*a workaround to solve cling issue*/
#include "../macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../load_libtorch.hpp"
/*import custom defined macros*/
#include "../custom_def.hpp"
/*import libtorch header file*/
#include <torch/torch.h>

***自动求梯度是PyTorch非常有特色的一项功能，不论是机器学习还是深度学习，我们经常需要对函数求梯度，特别是深度神经网络在执行反向传播操作时，求梯度是必备功能之一。***

***PyTorch提供的autograd包能够根据输入和前向传播过程自动构建计算图，并执行反向传播。***

# 1. 简单示例

下面先通过一些例子看看autograd功能是如何使用的(注：此处我们并不完全按照“dive into DL pytorch”教程中的方式来，而是结合了[官方教程](https://pytorch.org/tutorials/advanced/cpp_autograd.html))：

In [43]:
// std::cout << std::boolalpha;

//创建一个Tensor并设置requires_grad=true去跟踪其计算过程:
torch::Tensor x = torch::ones({2,2}, torch::requires_grad(true));
printT(x);

x = x+2;
//对于tensor加法而言，其梯度函数为：AddBackward1
printT(x.grad_fn()->name());


//如果上面参数torch::requires_grad设置为false，则上面语句会报错：
//null passed to a callee that requires a non-null argument [-Wnonnull]


auto y = x * x *3;
auto out = y.mean();

printT(y);
printT(y.grad_fn()->name());

printT(out);
printT(out.grad_fn()->name());

x = 
 1  1
 1  1
[ CPUFloatType{2,2} ]
<<--->>

x.grad_fn()->name() = 
AddBackward1
<<--->>

y = 
 27  27
 27  27
[ CPUFloatType{2,2} ]
<<--->>

y.grad_fn()->name() = 
MulBackward1
<<--->>

out = 
27
[ CPUFloatType{} ]
<<--->>

out.grad_fn()->name() = 
MeanBackward0
<<--->>



初始化Tensor后再修改其autograd属性：

In [40]:
std::cout << std::boolalpha;

auto a = torch::randn({2, 2});
a = ((a * 3) / (a - 1));
printT(a.requires_grad());

a.requires_grad_(true);
printT(a.requires_grad());

auto b = (a * a).sum();
printT(b.grad_fn()->name());


a.requires_grad() = 
false
<<--->>

a.requires_grad() = 
true
<<--->>

b.grad_fn()->name() = 
SumBackward0
<<--->>



In [53]:
//再回到刚开始那个例子，给out变量做一次反向传播运算，看看其梯度是多少
torch::Tensor x = torch::ones({2,2}, torch::requires_grad(true));
auto y = x+2;
auto z = y * y *3;
auto out = z.mean();
//求梯度，只能针对scalar类型的变量进行
out.backward();

printT(x.grad());

x.grad() = 
 4.5000  4.5000
 4.5000  4.5000
[ CPUFloatType{2,2} ]
<<--->>

